In [2]:
import os
import redis
import json
import numpy as np
from dask_gateway import GatewayCluster
cluster = GatewayCluster()
client = cluster.get_client()
redisHost = os.getenv("REDIS_HOST") if "REDIS_HOST" in os.environ else 'localhost'
cluster.scale(2)

### click the link in the field Dashboard: after running the command below

In [ ]:
cluster

In [21]:
def getdb():
    r = redis.Redis(host=redisHost,
                        port=6379, db=0)
    return r

In [22]:
#max string size is 512 mb
def dump_array(nparray):
    return json.dumps(nparray.tolist())
    
def decode_array(encStr):
    return np.asarray(json.loads(encStr))

def array_to_db(db, array, name):
    # we will store arrays as strings.
    db.set(name, dump_array(array))
    
def get_array(db, name):
    encStr = db.get(name)
    return decode_array(encStr)

In [23]:
db = getdb()

# Compute the SVD of a Tall-and-Skinny Matrix

In [24]:
import dask.array as da
X = da.random.random((2000000, 100), chunks=(10000, 100)).persist()

In [25]:
import dask

u, s, v = da.linalg.svd(X)


In [26]:
result = v.compute()

In [27]:
result

array([[ 0.10000143,  0.10003522,  0.09999744, ...,  0.09998082,
         0.10007407,  0.09999955],
       [-0.05414751,  0.13247819, -0.18712968, ..., -0.05485667,
        -0.02666097,  0.079013  ],
       [-0.00843602,  0.1909191 ,  0.0304151 , ..., -0.07306165,
         0.00371681,  0.04871533],
       ...,
       [ 0.13871317, -0.00167558, -0.12080768, ...,  0.11907913,
        -0.07532528, -0.03493167],
       [-0.09249286, -0.02106935, -0.06796025, ..., -0.11619282,
         0.05354037, -0.10194138],
       [ 0.06265146, -0.0564253 , -0.06042728, ...,  0.01978495,
        -0.10526863,  0.10554614]])

## put the result into the database

In [31]:
array_to_db(db,result,'demo2')

In [32]:
zstored = get_array(db,'demo2')

In [33]:
zstored

array([[ 0.10000143,  0.10003522,  0.09999744, ...,  0.09998082,
         0.10007407,  0.09999955],
       [-0.05414751,  0.13247819, -0.18712968, ..., -0.05485667,
        -0.02666097,  0.079013  ],
       [-0.00843602,  0.1909191 ,  0.0304151 , ..., -0.07306165,
         0.00371681,  0.04871533],
       ...,
       [ 0.13871317, -0.00167558, -0.12080768, ...,  0.11907913,
        -0.07532528, -0.03493167],
       [-0.09249286, -0.02106935, -0.06796025, ..., -0.11619282,
         0.05354037, -0.10194138],
       [ 0.06265146, -0.0564253 , -0.06042728, ...,  0.01978495,
        -0.10526863,  0.10554614]])